In [24]:
import numpy as np
import pandas as pd

In [25]:
# read csv and normalize rows names
excel = pd.read_csv('pagos.csv', encoding='latin-1', sep=',')
excel.columns = excel.columns.str.strip()
excel.head()

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo,%
0,09/06/2023,Manfry Ariza,"$ 60,000",Valoracion,No,Si,No,No,NaN
1,09/06/2023,Nazly Avila,"$ 185,000",Montaje/retiro,Si,No,No,No,NaN
2,09/06/2023,Paula Hernandez,"$ 25,000",Radiografia,No,No,No,Si,NaN
3,09/06/2023,Kevin Santiago Soto,"$ 105,000",Control/Reparaciones,No,No,No,Si,NaN
4,09/06/2023,Laura Cupa,"$ 60,000",Control,No,Si,No,No,NaN


In [26]:
excel.describe()

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo,%
count,10000,147,147,139,146,146,146,146,1
unique,244,120,43,46,2,2,2,2,1
top,17/07/2023,Atenais Peña,"$ 60,000",Control,No,No,No,Si,Sistecredito
freq,42,3,32,34,119,111,141,79,1


In [27]:
# Check missing values
excel.isna().sum()

Fecha               0
Paciente         9853
Valor            9853
Actividad        9861
Nequi            9854
Datafono         9854
Transferencia    9854
Efectivo         9854
%                9999
dtype: int64

As we can see, there is a lot of rows whith only the date, which means that they were just part of the excel template but they don't have any information. We can remove them.

In [28]:
# Delete rows with less than 2 non-NA values to delete rows with only the date
excel = excel.dropna(thresh=2)
print(f"Number of rows remaining: {len(excel)} \n")
print("Number of missing values per column:")
excel.isna().sum()

Number of rows remaining: 147 

Number of missing values per column:


Fecha              0
Paciente           0
Valor              0
Actividad          8
Nequi              1
Datafono           1
Transferencia      1
Efectivo           1
%                146
dtype: int64

In [29]:
# Delte "%" column as it only have 1 non-NA value
excel = excel.drop(columns='%')
print(f"Number of rows remaining: {len(excel)} \n")
print("Number of missing values per column:")
excel.isna().sum()

Number of rows remaining: 147 

Number of missing values per column:


Fecha            0
Paciente         0
Valor            0
Actividad        8
Nequi            1
Datafono         1
Transferencia    1
Efectivo         1
dtype: int64

As we only have a maximum of 12 rows with missing values, let's check them and see if we can fill them with the information from the previous row.

In [30]:
# Print the rows with NA values
excel[excel.isna().any(axis=1)]

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo
163,13/06/2023,Johana Martinez,"$ 20,000",NaN,Si,No,No,No
164,13/06/2023,Jeimy,"$ 30,000",NaN,No,No,No,Si
165,13/06/2023,Halam Diaz,"$ 58,500",NaN,No,No,No,Si
532,22/06/2023,Yeimy Quintero,"$ 60,000",NaN,No,No,No,Si
534,22/06/2023,Leimar Diaz,"$ 50,000",NaN,No,No,No,Si
536,22/06/2023,Paola Vargas,"$ 80,000",NaN,No,No,No,Si
538,22/06/2023,Cristian Ovalle,"$ 180,000",NaN,No,No,No,Si
9999,17/07/2023,Total desde 09/06/23,"$ 18,528,500",NaN,NaN,NaN,NaN,NaN


We can see that we have actually only 8 rows with missing values, from which 1 is a summary of the sales from certain period on the original excel file. We can remove it.

In [31]:
# Delte row of excel summary (index 9999)
excel = excel.drop(9999)
excel[excel.isna().any(axis=1)]

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo
163,13/06/2023,Johana Martinez,"$ 20,000",NaN,Si,No,No,No
164,13/06/2023,Jeimy,"$ 30,000",NaN,No,No,No,Si
165,13/06/2023,Halam Diaz,"$ 58,500",NaN,No,No,No,Si
532,22/06/2023,Yeimy Quintero,"$ 60,000",NaN,No,No,No,Si
534,22/06/2023,Leimar Diaz,"$ 50,000",NaN,No,No,No,Si
536,22/06/2023,Paola Vargas,"$ 80,000",NaN,No,No,No,Si
538,22/06/2023,Cristian Ovalle,"$ 180,000",NaN,No,No,No,Si


Now, before dealing with the missing values rest of the missing values, let's fix the data types of the columns.

In [32]:
# Convert "Valor" column into integer from the format "$ 12,345,678"
excel['Valor'] = excel['Valor'].str.replace('$', '')
excel['Valor'] = excel['Valor'].str.replace(',', '')
excel['Valor'] = excel['Valor'].str.replace(' ', '')
excel['Valor'] = excel['Valor'].astype(int)

excel.dtypes

Fecha            object
Paciente         object
Valor             int64
Actividad        object
Nequi            object
Datafono         object
Transferencia    object
Efectivo         object
dtype: object

In [33]:
# Convert "Si" and "No" to boolean
excel['Nequi'] = excel['Nequi'].replace({
    'Si': True,
    'No': False
}).astype('bool')
excel['Datafono'] = excel['Datafono'].replace({
    'Si': True,
    'No': False
}).astype('bool')
excel['Transferencia'] = excel['Transferencia'].replace({
    'Si': True,
    'No': False
}).astype('bool')
excel['Efectivo'] = excel['Efectivo'].replace({
    'Si': True,
    'No': False
}).astype('bool')

# Convert "Fecha" in format day/month/year to datetime
excel['Fecha'] = pd.to_datetime(excel['Fecha'], format='%d/%m/%Y')

# Standarize "Paciente" column
excel['Paciente'] = excel['Paciente'].str.strip().str.title()

Now, with the data types fixed, let's check the missing values again.


In [34]:
print("Types of columns:")
excel.dtypes

Types of columns:


Fecha            datetime64[ns]
Paciente                 object
Valor                     int64
Actividad                object
Nequi                      bool
Datafono                   bool
Transferencia              bool
Efectivo                   bool
dtype: object

With the data types fixed, for the 7 rows with missing values and 1 with invalid ones, they are all missing a proper value on the `Actividad` column. Therefore, we are going to check the percentage of the sales value that they represent to check if we can remove them temporarly.

In [35]:
# Create boolean indexing for rows with NA or invalid values in the 'Actividad' column
na_boolean_list = excel['Actividad'].isna()
# This is an invalid value as it was a note, not an activity
invalids_boolean_list = excel['Actividad'] == '90: Deuda'
nan_or_invalid = na_boolean_list | invalids_boolean_list

# Get the values of the rows with NA or invalid values in the 'Actividad' column
df_actividades_na = excel[nan_or_invalid]

total_na = df_actividades_na['Valor'].sum()

# Get the total amount of money
total = excel['Valor'].sum()

print(f'Total: ${total:,.2f} COP')
print(f'Total NA: ${total_na:,.2f} COP')
print(f'Lost money percentage: {total_na/total*100:.2f}%')

Total: $18,528,500.00 COP
Total NA: $508,500.00 COP
Lost money percentage: 2.74%


As they represent 2.74% of the total sales, we can remove them temporarly and check if we can fill them with the information in the future.

In [36]:
# Creating new database without nan/invalid values to clean and standardize activities names
cleaning_df = excel[~nan_or_invalid].copy()

Now, let's check the different values for activities to see if they are in a format easy to work with.

In [37]:
# Removing spaces at the start or the end
cleaning_df.loc[:, 'Actividad'] = cleaning_df['Actividad'].str.strip()

print(cleaning_df['Actividad'].unique())
print(f"\nTotal of activities: {len(cleaning_df['Actividad'].unique())}")

['Valoracion' 'Montaje/retiro' 'Radiografia' 'Control/Reparaciones'
 'Control' 'Bichectomia' 'Limpieza/braquets' 'Botox/Hilos' 'Faja'
 'Lipopapada y Bichectomia' 'Control Caucho' 'Limpieza' 'Reparaciones'
 'Reparacion Y Cauchos' 'Hilos' 'Lipopapada Bichectomia'
 'Tratamiento Odontologico' 'Calsa' 'Retenedores' 'Bichectomia Lipopapada'
 'Protesis' 'Limpieza Exodoncia' 'Control, Reparacion' 'Consulta General'
 'Control, Reparaciones' 'Control, Reparaciones, caucho' 'Lipopapada'
 'Control, Reparacion, Caucho' 'Control, Cauchos' 'Limpieza, Montaje'
 'Resina' 'Kit' 'Lipopada, Faja' 'Control, Reparaciones, Cauchos'
 'LipoPapada' 'Extraccion' 'Extracciones' 'Control, Exodoncia' 'Resinas'
 'Hilos, Retenedor' 'Controles, Reparacion' 'Papada, Encimas'
 'Placa Activada' 'Pago Porcentaje']

Total of activities: 44


As we can see, the activities format is not consistent. We can see that there are some activities that are repeated but with different names. Other's where there are more than one activity in the same row. And others where the activity is not clear.

Therefore, we need to standarize and clean the activities names.

In [38]:
# Remove all rows with activity name 'Pago Porcentaje',
# as they correspond to internal division of the payments and are not relevant for the analysis
cleaning_df = cleaning_df[~cleaning_df['Actividad'].str.
                          contains('Pago Porcentaje')]

# Define activities already in the correct format to avoid spliting them as different activities
exceptions = ['Tratamiento Odontologico', 'Consulta General', 'Placa Activada']

# Exclude exceptions from the data manipulation
df_clean_rows = cleaning_df[cleaning_df['Actividad'].isin(exceptions)]
cleaning_df = cleaning_df[~cleaning_df['Actividad'].isin(exceptions)]

print(f"Activities list size: {cleaning_df['Actividad'].unique().size}")
print(f"Non-cleaning data size: {df_clean_rows.shape}")
print(f"Cleaning data size: {cleaning_df.shape}")

Activities list size: 40
Non-cleaning data size: (3, 8)
Cleaning data size: (134, 8)


In [39]:
# Standardize the format of the activities by replacing different delimiters with a single comma
cleaning_df['Actividad'] = cleaning_df['Actividad'].str.replace(
    r'\s*[,/yY ]+\s*', ',', regex=True)

# Make everything lowercase
cleaning_df['Actividad'] = cleaning_df['Actividad'].str.lower()

print(cleaning_df['Actividad'].unique())
print(f"\nTotal of activities: {len(cleaning_df['Actividad'].unique())}")

['valoracion' 'montaje,retiro' 'radiografia' 'control,reparaciones'
 'control' 'bichectomia' 'limpieza,braquets' 'botox,hilos' 'faja'
 'lipopapada,bichectomia' 'control,caucho' 'limpieza' 'reparaciones'
 'reparacion,cauchos' 'hilos' 'calsa' 'retenedores'
 'bichectomia,lipopapada' 'protesis' 'limpieza,exodoncia'
 'control,reparacion' 'control,reparaciones,caucho' 'lipopapada'
 'control,reparacion,caucho' 'control,cauchos' 'limpieza,montaje' 'resina'
 'kit' 'lipopada,faja' 'control,reparaciones,cauchos' 'extraccion'
 'extracciones' 'control,exodoncia' 'resinas' 'hilos,retenedor'
 'controles,reparacion' 'papada,encimas']

Total of activities: 37


In [40]:
# Replace conjugations of activities name with only one cunjugation

replacements = {
    r'\bcontroles\b': 'control',
    r'\breparaciones\b': 'reparacion',
    r'\bextracciones\b': 'extraccion',
    r'\bcaucho\b': 'cauchos',
    r'\blipopada\b': 'lipopapada',
    r'\bresinas\b': 'resina',
    r'\bretenedor\b': 'retenedores'
}

# Apply replacements
cleaning_df['Actividad'] = cleaning_df['Actividad'].replace(replacements,
                                                            regex=True)

print(cleaning_df['Actividad'].unique())
print(f"\nTotal of activities: {len(cleaning_df['Actividad'].unique())}")

['valoracion' 'montaje,retiro' 'radiografia' 'control,reparacion'
 'control' 'bichectomia' 'limpieza,braquets' 'botox,hilos' 'faja'
 'lipopapada,bichectomia' 'control,cauchos' 'limpieza' 'reparacion'
 'reparacion,cauchos' 'hilos' 'calsa' 'retenedores'
 'bichectomia,lipopapada' 'protesis' 'limpieza,exodoncia'
 'control,reparacion,cauchos' 'lipopapada' 'limpieza,montaje' 'resina'
 'kit' 'lipopapada,faja' 'extraccion' 'control,exodoncia'
 'hilos,retenedores' 'papada,encimas']

Total of activities: 30


Now, we are going to split the activities that have more than one activity in the same row.

In [41]:
# Get the list of different single activities
activities = cleaning_df['Actividad'].str.split(',',
                                                expand=True).stack().unique()

print(f"Unique single activities: {activities}")
print(f"\nTotal of single activities: {len(activities)}")

Unique single activities: ['valoracion' 'montaje' 'retiro' 'radiografia' 'control' 'reparacion'
 'bichectomia' 'limpieza' 'braquets' 'botox' 'hilos' 'faja' 'lipopapada'
 'cauchos' 'calsa' 'retenedores' 'protesis' 'exodoncia' 'resina' 'kit'
 'extraccion' 'papada' 'encimas']

Total of single activities: 23


In [42]:
# Split the activities by comma into new columns
activities_df = cleaning_df['Actividad'].str.split(
    ',', expand=True).rename(columns=lambda x: f'Actividad_{x + 1}')

# Join the new columns with the main dataframe into a new rebuilt one
rebuilt_df = cleaning_df.join(activities_df, sort=True)

rebuilt_df.columns.to_list()

['Fecha',
 'Paciente',
 'Valor',
 'Actividad',
 'Nequi',
 'Datafono',
 'Transferencia',
 'Efectivo',
 'Actividad_1',
 'Actividad_2',
 'Actividad_3']

Now, we are going to join the dataframe with the cleaned activities names with the one of the exceptions.

In [43]:
# Add "Actividad_1" column to the non-cleaning data
df_clean_rows['Actividad_1'] = df_clean_rows['Actividad']
# Append the non-cleaning data to the rebuilt one
rebuilt_df = pd.concat([rebuilt_df, df_clean_rows])

# Rename column Actividad to Actividades
rebuilt_df = rebuilt_df.rename(columns={'Actividad': 'Actividades'})

# Reorder columns
columns = rebuilt_df.columns.tolist()
columns = columns[:4] + columns[-3:] + columns[4:8]

rebuilt_df = rebuilt_df[columns]

print(f"Data size: {len(rebuilt_df)}")
rebuilt_df.head()

Data size: 137


,Fecha,Paciente,Valor,Actividades,Actividad_1,Actividad_2,Actividad_3,Nequi,Datafono,Transferencia,Efectivo
0,2023-06-09,Manfry Ariza,60000,valoracion,valoracion,None,None,False,True,False,False
1,2023-06-09,Nazly Avila,185000,"montaje,retiro",montaje,retiro,None,True,False,False,False
2,2023-06-09,Paula Hernandez,25000,radiografia,radiografia,None,None,False,False,False,True
3,2023-06-09,Kevin Santiago Soto,105000,"control,reparacion",control,reparacion,None,False,False,False,True
4,2023-06-09,Laura Cupa,60000,control,control,None,None,False,True,False,False


In [44]:
# Check that there are only missing values for columns Actividad_2 and Actividad_3
rebuilt_df.isna().sum()

Fecha              0
Paciente           0
Valor              0
Actividades        0
Actividad_1        0
Actividad_2       87
Actividad_3      133
Nequi              0
Datafono           0
Transferencia      0
Efectivo           0
dtype: int64

Finally, we create csv files for the cleaned dataframe and the one with the missing activities we ommited.

In [45]:
# Convert cleaned data to csv
rebuilt_df.to_csv('pagos_cleaned.csv', index=True, index_label='Index')
rebuilt_df.head()

,Fecha,Paciente,Valor,Actividades,Actividad_1,Actividad_2,Actividad_3,Nequi,Datafono,Transferencia,Efectivo
0,2023-06-09,Manfry Ariza,60000,valoracion,valoracion,None,None,False,True,False,False
1,2023-06-09,Nazly Avila,185000,"montaje,retiro",montaje,retiro,None,True,False,False,False
2,2023-06-09,Paula Hernandez,25000,radiografia,radiografia,None,None,False,False,False,True
3,2023-06-09,Kevin Santiago Soto,105000,"control,reparacion",control,reparacion,None,False,False,False,True
4,2023-06-09,Laura Cupa,60000,control,control,None,None,False,True,False,False


In [46]:
# Convert invalid values into nan before converting to csv
df_actividades_na.loc[:, 'Actividad'] = np.nan

# Convert into csv
df_actividades_na.to_csv('pagos_na.csv', index=True, index_label='Index')

df_actividades_na

,Fecha,Paciente,Valor,Actividad,Nequi,Datafono,Transferencia,Efectivo
163,2023-06-13,Johana Martinez,20000,NaN,True,False,False,False
164,2023-06-13,Jeimy,30000,NaN,False,False,False,True
165,2023-06-13,Halam Diaz,58500,NaN,False,False,False,True
532,2023-06-22,Yeimy Quintero,60000,NaN,False,False,False,True
534,2023-06-22,Leimar Diaz,50000,NaN,False,False,False,True
535,2023-06-22,Leimar Diaz,30000,NaN,True,False,False,False
536,2023-06-22,Paola Vargas,80000,NaN,False,False,False,True
538,2023-06-22,Cristian Ovalle,180000,NaN,False,False,False,True
